BAYESIAN INFERENCE AND MODEL SELECTION ON PL DATA USING NESTED SAMPLING

This notebook is used to perform inference on photoluminescence data of lead halide perovskites using a (flexible) physical model. The aim is to return both a posterior distribution for the physical parameters used in the model, and the total evidence obtained by the sampling to be used for model comparison.

Created by Florine Rombach.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path, PurePath
import matplotlib.pyplot as plt
import sys
from matplotlib import colormaps as cm
import dynesty
import dynesty.utils
import dill
dynesty.utils.pickle_module = dill
import datetime
from dynesty import plotting as dyplot

from import_data import *
from model import *
from nested_sampling import *
from plotting import *

rstate = np.random.default_rng(56101)
np.set_printoptions(threshold=sys.maxsize)

THE DATA

Inference is performed on data from two different measurements of a perovskite thin film, performed on multiple samples with slightly different growth conditions:

- Time-resolved photoluminescence, in which a single laser pulse is used to generate      electron-hole pairs in the material and the photoluminescence resulting from the radiative recombination of these charge carriers is measured over time. The photoluminescence decays over time due to charge recombination.

- Photoluminescence quantum efficiency, in which a continuous-wave laser is used to generate electron-hole pairs in the material and the photoluminescence resulting from the radiative recombination of these charge carriers is measured at steady-state equilibrium. The calculated quantum efficiency reflects the rate of radiative to total recombination, attenuated by photon reabsorption.

From these measurements, we would like to get information about the rates of various difference recombination processes in our material.

In [2]:
# An input file is used to supply measurement and sample information.
# Choose folder in which the input file is saved. Output files will also be saved here.
directory = Path('./project_data')  
database_path = directory / 'input-all.txt'             
input_df = pd.read_csv(database_path, sep='\t')

# Provide the nr of processes to split the nested sampling into (for parallelization).
nr_proc = 4

# Provide the hyperparameters for nested sampling
nr_live_points = 256 # (rule of thumb: at least 2*ndim and 50 per expected posterior mode)
bound = 'multi' # allows for multimodal posterior distribution
sample = 'rslice' # effective gradient-free sampling method for this problem
noise_threshhold=2

data = import_all_data(directory, input_df, trpl_laser_reference_file, noise_threshhold, plqe_laser_reference_file)

THE MODEL

Very briefly, the model used in this example is editable but currently includes Auger recombination, radiative recombination with 95% photon reabsorption, non-radiative recombination via one electron and one hole trap with variable energy levels, and a shiftable Fermi level. The variation in the density of charge carriers over time or at steady state under these conditions is described by systems of equations in model.py.

Spatially resolved differences in carrier density (stemming from inhomogenous distribution of traps, surface recombination and/or low mobility) are not currently included.

In [ ]:
# Define parameters 

# Define [name, initial value (for visualization only), lower bound, upper bound, distribution type, vary or keep constant]
# Note: lognormally distributed parameters are already rescaled as log(param)

# Initial parameter values have no impact on fit, and are only for visualization
# lbound and upound represent 95% confidence range for normal dist, and bounds for flat distributions

samples = list(dict.fromkeys(input_df['sample']))

param_dict = defaultdict(dict)

for i in range(len(samples)):

    Eg = list(input_df.loc[input_df['sample']==samples[i], 'bandgap (eV)'])[0] # import sample bandgap to help set bounds

    #                                             PARAM NAME (for postproc)         INITIAL LBOUND  UBOUND      DISTRIBUTION VARY?
    param_dict[samples[i]]['p_Ef'] = (            [f'E fermi {samples[i]}',         0.2,    0.025,  Eg-0.025,   'flat',     'constant'])       # Fermi level of perovskite

    param_dict[samples[i]]['p_Ca'] = (            [f'log kaug {samples[i]}',        -28,    -28.5,  -26.5,      'normal',   'vary together'])  # EXP! cm6 s-1 (auger coefficient for nnp)

    param_dict[samples[i]]['p_krad'] = (          [f'log krad {samples[i]}',        -10,    -11,    -9,         'normal',   'vary together'])  # EXP! cm3 s-1 (internal radiative recombination coefficient)
    param_dict[samples[i]]['p_Pa'] = (            [f'Pa {samples[i]}',              0,      0,      1,          'flat',     'constant'])       # probability of parasitic absorption
    param_dict[samples[i]]['p_Pr'] = (            [f'Pr {samples[i]}',              0.95,   0,      1,          'flat',     'constant'])       # probability of reabsorption by perovskite

    param_dict[samples[i]]['p_N_1'] = (           [f'log N trap1 {samples[i]}',     14,     10,     20,         'flat',     'vary indv'])      # EXP! cm-3 (trap density)
    param_dict[samples[i]]['p_depth_trap_1'] = (  [f'E trap1 {samples[i]}',         1.02,   Eg/2,    Eg-0.025,  'flat',     'vary together'])  # eV (distance of trap from VB)
    param_dict[samples[i]]['p_bp_1'] = (          [f'log beta p trap1 {samples[i]}', -9,    -12,    -6,         'flat',     'vary together'])  # EXP! (p capture coefficient)
    param_dict[samples[i]]['p_bn_1'] = (          [f'log beta n trap1 {samples[i]}', -9,    -12,    -6,         'flat',     'constant'])       # EXP! (n capture coefficient)

    param_dict[samples[i]]['p_N_2'] = (           [f'log N trap2 {samples[i]}',      17,    10,     20,         'flat',     'constant'])      # EXP! cm-3 (trap density)
    param_dict[samples[i]]['p_depth_trap_2'] = (  [f'E trap2 {samples[i]}',         0.05,   0.025,   Eg/2,      'flat',     'constant'])       # eV (distance of trap from VB)
    param_dict[samples[i]]['p_bn_2'] = (          [f'log beta n trap2 {samples[i]}', -9,    -12,    -6,         'flat',     'constant'])  # EXP! (n capture coefficient)
    param_dict[samples[i]]['p_bp_2'] = (          [f'log beta p trap2 {samples[i]}', -9,    -12,    -6,         'flat',     'constant'])       # EXP! (p capture coefficient)

    param_dict[samples[i]]['p_A_ct'] = (          [f'Act {samples[i]}',               0,    0,      0.4,        'flat',     'constant'])       # scaling of delay (allows begin lower than 1)

# Initial guess plot
param_input_in = {}
param_input_in[samples[0]] = {key: entry[1] for key, entry in param_dict[samples[0]].items() 
                              if entry[5] =='vary indv' or entry[5] =='vary together'}
for i in range(1, len(samples)):
    param_input_in[samples[i]] = {key: entry[1] for key, entry in param_dict[samples[i]].items() 
                                  if entry[5] =='vary indv'}

plot_fit(directory, samples, input_df, data, param_dict, mode = 'single',
         param_input_in = param_input_in,
         samples_reweighted = None, param_vary_keys = None, param_vary_list = None, nfits = None)

BAYESIAN INFERENCE

Bayesian inference uses a likelihood function (which describes how well the model describes the data for a set of parameters) and a collection of prior distributions (which contain previous knowledge about the set of parameters) to calculate posterior distributions for each parameter. This posterior describes what we can know about the model parameters given our previous knowledge and our data.
 
posterior   =   prior  * likelihood / evidence
p(θ|D)      =   ( p(θ) * p(D|θ) ) / ∫ p(D|θ) p(θ) dθ

Whilst Markov Chain Monte Carlo (MCMC) sampling methods manage posterior estimation by avoiding computation of the evidence, Nested Sampling methods (introduced by Skilling, 2006) compute the evidence directly. This is useful as quantifying the total evidence allows for Bayesian model comparison, which can help make decisions about which model should be used to describe data.
                  
                  ratio of evidence       ratio of model prior (often 1)
Bayes factor = ( p(M2|D) / p(M1|D) ) * ( p(M2) / p(M1) )

The dynesty package (Josh Speagle and contributors) used here further allows for 'Dynamic Nested Sampling', in which samples are added after the completion of basic nested sampling to improve the posterior determination. This is not currently used in this example notebook.

In [4]:
# Initialize dynamic nested sampler

# Define the list of parameters to vary according to description above
param_vary = {key+'-'+samples[0]: entry for key, entry in param_dict[samples[0]].items() 
              if entry[5] =='vary indv' or entry[5] =='vary together'}
for i in range(1, len(samples)):
    param_vary.update({key+'-'+samples[i]: entry for key, entry in param_dict[samples[i]].items() 
                       if entry[5] =='vary indv'})

param_vary_list = list(param_vary.values())
param_vary_keys = list(param_vary.keys())
ndim = len(param_vary_keys)

# Make a new savepath for every sampling run started.
savepath = directory / ('run_'+((str(datetime.datetime.now()).replace(" ", "_")).replace(":", "_")))
savepath.mkdir(exist_ok=True)
checkpoint_file = PurePath(savepath / 'nested_sampling_run_backup.save')
print(checkpoint_file)

# Run nested sampler in parallel.
with dynesty.pool.Pool(nr_proc, log_likelihood, prior_transform, logl_args=(param_vary_keys, param_dict, input_df, data), ptform_kwargs=({'param_vary_list': param_vary_list}) ) as pool:

    sampler = dynesty.DynamicNestedSampler(pool.loglike, pool.prior_transform, ndim = ndim,
                                rstate = rstate,
                                bound = bound, #options for bounding the target distribution
                                nlive = nr_live_points, # nr of live points being sampled from (default 250, at least 2*ndim and 50 per expected mode)
                                sample = sample, # sampling method conditioned on the provided bounds
                                pool = pool
                                )

    sampler.run_nested(checkpoint_file = str(checkpoint_file))

project_data/run_2024-07-21_19_44_50.067085/nested_sampling_run_backup.save


2458it [1:56:28,  5.08s/it, batch: 0 | bound: 20 | nc: 134 | ncall: 82691 | eff(%):  2.963 | loglstar:   -inf < 1174.996 <    inf | logz: 1162.242 +/-  0.213 | dlogz: 55.658 >  0.010] 

In [ ]:
# optionally: restore sampler object if not in memory
# sampler = dynesty.DynamicNestedSampler.restore(str(checkpoint_file))

results = sampler.results
results.summary()

# Post-processing saves sampler parameters, all accepted samples, 
# and dynesty's default plots describing the sampler progress, sampled space for each parameter, and posterior cornerplots.
samples_reweighted = post_run_info(results, nr_live_points, bound, sample, savepath, param_vary_list)


# Plot multi-draw overlay of model fit onto data
# NOTE: this currently assumes that the posterios distributions are all gaussian
plot_fit(savepath, samples, input_df, data, param_dict, mode = 'dist', 
         param_input_in = None, 
         samples_reweighted = samples_reweighted, param_vary_list = param_vary_list, param_vary_keys = param_vary_keys, nfits = 100)